In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as spd
import psColor, bwLabel
import os

# Load training dataset
TRAIN_DIR = os.listdir(path='../data/treino')
DATA = [cv2.imread('../data/treino/' + image_path) for image_path in TRAIN_DIR]

print(f"Number of images in training set: {len(DATA)}")
print(f"Image shape: {DATA[0].shape}")

## 1. Utility Functions

In [ ]:
def showImages(imageArray, titles):
    """Display de images com OpenCV."""
    for i, img in enumerate(imageArray):
        cv2.imshow(titles[i], img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def pltImages(imageArray, titles, cmap=None):
    """Display de imagens com Matplotlib subplots."""
    n_images = len(imageArray)
    for i, image in enumerate(imageArray):
        plt.subplot(n_images, 1, i + 1)
        plt.imshow(image, cmap=cmap)
        plt.title(titles[i])
    plt.tight_layout()

## 2. Processamento de Imagem

In [ ]:
def binarizacao(image, tolerance):
    """
    Perform image binarization to highlight objects by removing the background.
    
    Args:
        image (np.array): BGR image array to be binarized
        tolerance (int): Tolerance for calculating the background color range
        
    Returns:
        tuple: (image_objects, bin_image)
            - image_objects: RGB image without background
            - bin_image: Binary image
    """
    # Calculate histogram for each color channel
    hist_blue = cv2.calcHist([image], [0], None, [256], [0, 256])
    max_blue_idx = np.argmax(hist_blue)
    low_blue = max_blue_idx - tolerance
    high_blue = max_blue_idx + tolerance

    hist_green = cv2.calcHist([image], [1], None, [256], [0, 256])
    max_green_idx = np.argmax(hist_green)
    low_green = max_green_idx - 60
    high_green = max_green_idx + 60

    hist_red = cv2.calcHist([image], [2], None, [256], [0, 256])
    max_red_idx = np.argmax(hist_red)
    low_red = max_red_idx - 40
    high_red = max_red_idx + 40

    # Define color range for background
    low_color = np.array([low_blue, low_green, low_red])
    high_color = np.array([high_blue, high_green, high_red])
    
    # Remove background
    background_mask = cv2.inRange(image, low_color, high_color)
    inverted_mask = 255 - background_mask
    image_objects = cv2.bitwise_and(image, image, mask=inverted_mask)
    
    # Convert to grayscale and apply Otsu's thresholding
    image_gs = cv2.cvtColor(image_objects, cv2.COLOR_BGR2GRAY)
    _, bin_image = cv2.threshold(image_gs, 0, 255, cv2.THRESH_OTSU)
    
    return image_objects, bin_image

def optimize_image(image):
    """
    Apply morphological operations to improve binary image quality.
    
    Args:
        image (np.array): Binary image to optimize
        
    Returns:
        np.array: Optimized binary image
    """
    image_opt = image
    
    # Close small gaps
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    image_opt = cv2.morphologyEx(image_opt, cv2.MORPH_CLOSE, kernel)
    
    # Erode to separate touching objects
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (50, 50))
    image_opt = cv2.morphologyEx(image_opt, cv2.MORPH_ERODE, kernel)
    
    # Dilate to restore object size to approximate original
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (25, 25))
    image_opt = cv2.morphologyEx(image_opt, cv2.MORPH_DILATE, kernel)
    
    return image_opt

## 3. Extração de Componentes

In [ ]:
def get_contours(img):
    """Extract contours from an image."""
    _, bin_img = binarizacao(img, 110)
    opt_img = optimize_image(bin_img)
    contours, _ = cv2.findContours(opt_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def get_propriedades_container(container):
    """
    Extract geometric properties from a list of contours.
    
    Properties extracted:
    - Area
    - Perimeter
    - Circularity
    - Delta (difference between min area rect and actual area)
    - Proportion (width/height ratio)
    
    Args:
        container (list): List of contours
        
    Returns:
        np.array: Array of shape (n_contours, 5) containing properties
    """
    props_container = np.zeros((len(container), 5))
    
    for i, cnt in enumerate(container):
        area = cv2.contourArea(cnt)
        per = cv2.arcLength(cnt, True)
        circ = 4 * np.pi * area / per ** 2
        
        min_rect_area = cv2.minAreaRect(cnt)
        width = min_rect_area[1][0]
        height = min_rect_area[1][1]
        delta = (width * height) - area
        propor = width / height if height > 0 else 0
        
        props = [area, per, circ, delta, propor]
        props_container[i] = props
        
    return props_container

def extracao_propriedades(imagem_otimizada):
    """
    Extract properties from all objects in an optimized binary image.
    
    Args:
        imagem_otimizada (np.array): Optimized binary image
        
    Returns:
        np.array: Properties of all detected objects
    """
    contours, _ = cv2.findContours(imagem_otimizada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    container = list(contours)
    return get_propriedades_container(container)

def get_estatistica_propriedades(vetor_props):
    """Calculate mean and standard deviation of properties."""
    media_props = np.mean(vetor_props, axis=0)
    desvio_props = np.std(vetor_props, axis=0)
    return media_props, desvio_props

## 4. Preparação da data de treino

In [ ]:
# Initialize containers for each brick type
container_2x2 = []
container_2x4 = []
container_2x6 = []
container_2x8 = []
container_undef = []

def insert_in_container(img, container, contour_idx):
    """Add specific contours to a container."""
    contours = get_contours(img)
    for i in contour_idx:
        container.append(contours[i])

### Labeling Manual da data de treino

In [ ]:
# Image 0: 
# 2x2: [0,7,8], 
# 2x4: [2,4,5], 
# 2x8: [6], 
# Undef: [1,3]
insert_in_container(DATA[0], container_2x2, [0, 7, 8])
insert_in_container(DATA[0], container_2x4, [2, 4, 5])
insert_in_container(DATA[0], container_2x8, [6])
insert_in_container(DATA[0], container_undef, [1, 3])

# Image 1: 
# 2x2: [4,5], 
# 2x4: [0,2], 
# 2x8: [3], 
# Undef: [1]
insert_in_container(DATA[1], container_2x2, [4, 5])
insert_in_container(DATA[1], container_2x4, [0, 2])
insert_in_container(DATA[1], container_2x8, [3])
insert_in_container(DATA[1], container_undef, [1])

# Image 2: 
# 2x2: [0,2], 
# 2x4: [1]
insert_in_container(DATA[2], container_2x2, [0, 2])
insert_in_container(DATA[2], container_2x4, [1])

# Image 3: 
# 2x4: [0,1], 
# 2x6: [2], 
# 2x8: [3]
insert_in_container(DATA[3], container_2x4, [0, 1])
insert_in_container(DATA[3], container_2x6, [2])
insert_in_container(DATA[3], container_2x8, [3])

# Image 4: 
# 2x4: [1], 
# 2x8: [2], 
# Undef: [0]
insert_in_container(DATA[4], container_2x4, [1])
insert_in_container(DATA[4], container_2x8, [2])
insert_in_container(DATA[4], container_undef, [0])

# Image 5: 
# 2x4: [0,2], 
# 2x8: [1]
insert_in_container(DATA[5], container_2x4, [0, 2])
insert_in_container(DATA[5], container_2x8, [1])

# Image 6: 
# Undef: [0,1,2]
insert_in_container(DATA[6], container_undef, [0, 1, 2])

# Image 7: 
# 2x4: [0,1], 
# 2x6: [2], 
# Undef: [3]
insert_in_container(DATA[7], container_2x4, [0, 1])
insert_in_container(DATA[7], container_2x6, [2])
insert_in_container(DATA[7], container_undef, [3])

# Image 8: 
# 2x2: [0], 
# 2x4: [1], 
# 2x6: [2], 
# 2x8: [3]
insert_in_container(DATA[8], container_2x2, [0])
insert_in_container(DATA[8], container_2x4, [1])
insert_in_container(DATA[8], container_2x6, [2])
insert_in_container(DATA[8], container_2x8, [3])

# Image 9: 
# 2x2: [0,1,2,3]
insert_in_container(DATA[9], container_2x2, [0, 1, 2, 3])

# Image 10: 
# 2x4: [0,1,2,3]
insert_in_container(DATA[10], container_2x4, [0, 1, 2, 3])

# Image 11: 
# 2x8: [0]
insert_in_container(DATA[11], container_2x8, [0])

# Image 12: 
# 2x2: [1], 
# 2x4: [0], 
# 2x6: [3,4], 
# Undef: [2]
insert_in_container(DATA[12], container_2x2, [1])
insert_in_container(DATA[12], container_2x4, [0])
insert_in_container(DATA[12], container_2x6, [3, 4])
insert_in_container(DATA[12], container_undef, [2])

# Image 13: 
# 2x2: [1], 
# 2x4: [0], 
# 2x6: [4], 
# Undef: [2,3]
insert_in_container(DATA[13], container_2x2, [1])
insert_in_container(DATA[13], container_2x4, [0])
insert_in_container(DATA[13], container_2x6, [4])
insert_in_container(DATA[13], container_undef, [2, 3])

print(f"Training samples \n2x2: {len(container_2x2)}, \n2x4: {len(container_2x4)}, "
      f"\n2x6: {len(container_2x6)}, \n2x8: {len(container_2x8)}, \nUndefined: {len(container_undef)}")

### Calcular propriedades e estatísticas para cada classe

In [ ]:
# Calculate properties and statistics for each brick type
props_2x2 = get_propriedades_container(container_2x2)
media_2x2, desvio_2x2 = get_estatistica_propriedades(props_2x2)

props_2x4 = get_propriedades_container(container_2x4)
media_2x4, desvio_2x4 = get_estatistica_propriedades(props_2x4)

props_2x6 = get_propriedades_container(container_2x6)
media_2x6, desvio_2x6 = get_estatistica_propriedades(props_2x6)

props_2x8 = get_propriedades_container(container_2x8)
media_2x8, desvio_2x8 = get_estatistica_propriedades(props_2x8)

### Gráficos de Distribuição das propriedades

In [ ]:
plt.figure(figsize=(15, 10))
property_names = ['Area', 'Perimeter', 'Circularity', 'Delta', 'Proportion']

for i in range(5):
    plt.subplot(3, 2, i + 1)
    plt.title(property_names[i], fontsize=12, fontweight='bold')
    plt.hist(props_2x2[:, i], bins=15, alpha=0.5, label='2x2')
    plt.hist(props_2x4[:, i], bins=15, alpha=0.5, label='2x4')
    plt.hist(props_2x6[:, i], bins=15, alpha=0.5, label='2x6')
    plt.hist(props_2x8[:, i], bins=15, alpha=0.5, label='2x8')
    plt.legend()
    plt.xlabel('Value')
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## 5. Classificação

In [ ]:
def distance_based_classification(propriedades, debug=False, std_deviation_elimination=False, confidence_threshold=0.5):
    """
    Classify objects based on Euclidean distance to class means.
    
    Objects are classified as undefined if their distance exceeds
    2 times the maximum standard deviation of the nearest class.
    
    Args:
        propriedades (np.array): Array of object properties
        
    Returns:
        np.array: Classification results [width, length] for each object
    """
    results = np.zeros((len(propriedades), 2))
    
    desvio_dict = {
        2: desvio_2x2,
        4: desvio_2x4,
        6: desvio_2x6,
        8: desvio_2x8
    }
    
    for i, vetor_prop in enumerate(propriedades):
        # Calculate Euclidean distance to each class mean
        distances = {
            2: spd.euclidean(vetor_prop, media_2x2),
            4: spd.euclidean(vetor_prop, media_2x4),
            6: spd.euclidean(vetor_prop, media_2x6),
            8: spd.euclidean(vetor_prop, media_2x8)
        }
        
        # Find closest class
        # predict_class = min(distances, key=distances.get)
        sorted_dist = sorted(distances.items(), key=lambda x: x[1])
        predict_class = sorted_dist[0][0]
        second_closest_class = sorted_dist[1][0]
        
        if debug:
            print(f"Object {i}: Distances {distances}, Predicted class: {predict_class}")
            print(f"desvio_dict[predict_class].max() * 2: {desvio_dict[predict_class].max() * 2}")
            print(f"Confidence threshold: {confidence_threshold * distances[predict_class]}")
            print(f"distances[second_closest_class] - distances[predict_class]: {distances[second_closest_class] - distances[predict_class]}")
            
        # Verificar se distancia excede o desvio padrão por duas vezes (Se std_deviation_elimination=True)
        if (distances[predict_class] > desvio_dict[predict_class].max() * 2) and std_deviation_elimination:
            predict_class = 0
        # Eliminar classificações com baixa confiança
        elif distances[second_closest_class] - distances[predict_class] < confidence_threshold * distances[predict_class]:
            predict_class = 0
            
        results[i] = np.array([2, predict_class])
        
    return results

## 6. Testes

In [ ]:
def process_and_classify(image, tolerance=110, show_images=True, debug=False, std_deviation_elimination=False, confidence_threshold=0.5):
    """
    Complete pipeline: preprocess, optimize, extract features, and classify.
    
    Args:
        image (np.array): Input BGR image
        tolerance (int): Tolerance for binarization
        show_images (bool): Whether to display intermediate results
        
    Returns:
        np.array: Classification results
    """
    # Preprocessing
    image_objects, bin_image = binarizacao(image, tolerance)
    image_opt = optimize_image(bin_image)
    
    # Feature extraction and classification
    propriedades = extracao_propriedades(image_opt)
    resultados = distance_based_classification(propriedades, debug=debug, std_deviation_elimination=std_deviation_elimination, confidence_threshold=confidence_threshold)
    
    # Display results if requested
    if show_images:
        showImages([image, image_objects, bin_image, image_opt],
                  ["Original", "No Background", "Binary", "Optimized"])
    
    return resultados

### Teste 1: Imagem 7
Expected: [2x4, 2x4, 2x6, Undefined]

In [ ]:
print("Test Case 1 - Image 7")
print("Expected: 2x4, 2x4, 2x6, Undefined")
resultados_teste1 = process_and_classify(DATA[7], show_images=False, debug=False, std_deviation_elimination=True, confidence_threshold=0.5)
print("Results:")
print(resultados_teste1)

### Teste 2: Imagem 4
Expected: [Undefined, 2x4, 2x8]

In [ ]:
print("Test Case 2 - Image 4")
print("Expected: Undefined, 2x4, 2x8")
resultados_teste2 = process_and_classify(DATA[4], show_images=False, debug=False, std_deviation_elimination=True, confidence_threshold=0.5)
print("Results:")
print(resultados_teste2)

### Teste 3: Imagem 0
Expected: [2x2, Undefined, 2x4, Undefined, 2x4, 2x4, 2x8, 2x2, 2x2]

In [ ]:
print("Test Case 3 - Image 0")
print("Expected: 2x2, Undefined, 2x4, Undefined, 2x4, 2x4, 2x8, 2x2, 2x2")
resultados_teste3 = process_and_classify(DATA[0], show_images=False, debug=False, std_deviation_elimination=True, confidence_threshold=0.5)
print("Results:")
print(resultados_teste3)

### Teste 4: Imagem 3
Expected: [2x4, 2x4, 2x6, 2x8]

In [ ]:
print("Test Case 4 - Image 3")
print("Expected: 2x4, 2x4, 2x6, 2x8")
resultados_teste4 = process_and_classify(DATA[3], show_images=False, debug=False, std_deviation_elimination=True, confidence_threshold=0.5)
print("Results:")
print(resultados_teste4)

### Teste 5: Imagem 12
Expected: [2x4, 2x2, Undefined, 2x6, 2x6]

In [ ]:
print("Test Case 5 - Image 12")
print("Expected: 2x4, 2x2, Undefined, 2x6, 2x6")
resultados_teste5 = process_and_classify(DATA[12], show_images=False, debug=False, std_deviation_elimination=True, confidence_threshold=0.1)
print("Results:")
print(resultados_teste5)